In [23]:
libOfInterest = 'safe_app'

In [30]:
libOfInterest = 'safe_authenticator'

In [31]:
with open('../safenet/extracted_headers/' + libOfInterest + '_function_declarations') as f:
    functionDeclarations=f.read()

In [32]:
oneline=functionDeclarations.splitlines()[3]

In [33]:
import re

In [34]:
startOfFunction=re.compile('.*?\(')

parameters=re.compile('\(.*\)')

functions=re.compile(', void \(\*.*?\)\(.*?\)')

In [35]:
for oneline in [(lambda x:x)(x) for x in functionDeclarations.splitlines() if x != '']:
    nameOfFunction=startOfFunction.findall(oneline)[0][5:-1]
    allParameters=parameters.findall(oneline)[0][1:-1]
    callbackfunctions=functions.findall(oneline)
    CallParameters=allParameters

    for oneCallback in callbackfunctions:
        CallParameters=CallParameters.replace(oneCallback,'')

    def getParameterDict(callParameters):
        mydict={}
        for onePar in callParameters:
            datatype,name = onePar.split()
            mydict[name]=datatype
        return mydict

    callbackDict={}
    for oneCallback in callbackfunctions:
        callbackDict[re.findall('\(\*.*?\)',oneCallback)[0][2:-1]]=getParameterDict(re.findall('\(.*?\)',oneCallback.replace(' const',''))[-1][1:-1].split(', '))

    def getFunctionDefPy(name,parameters,praefix,callbackNames):
        line=praefix + 'def ' + '_'+name + '(self, '
        innerStuff=''
        for x in [(lambda x: x.split()[-1])(x) for x in parameters.split(',')]:
             innerStuff += x + ', '
        innerStuff = innerStuff[:-2]
        line = line + innerStuff 
        for one in callbackNames:
            line += ', ' + one+'=None'
        line = line +', safenetLib=self.lib.' + libOfInterest + '):'
        return line,innerStuff

    def getFunctionCommentPy(name,parameters,praefix):
        line=praefix + 'def '+name + '('
        for x in [(lambda x: x.split()[0])(x) for x in parameters.split(',')]:
            line += x + ', '
        line = line[:-2]+', safenetLib=self.lib.' + libOfInterest + '):'
        return line

    def printMethodDefinition(callParameters_string,callbackfunctions,callbackNames):

        types = [(lambda x: x.split()[0])(x) for x in callParameters_string.split(',')]
        names = [(lambda x: x.split()[-1])(x) for x in callParameters_string.split(',')]
        callParameters={}
        for i,currType in enumerate(types):
            callParameters[names[i]]=currType

        #now the doc string
        tempstring='    """\n        '
        for onepar in callParameters:
            if callParameters[onepar] == 'void*':
                tempstring += '[any]' + ', '
            elif callParameters[onepar] == 'char*':
                tempstring += 'bytes' + ', '
            else:
                tempstring += callParameters[onepar] + ', '
        for onepar in callbackfunctions:
            tempstring += '[function], '

        print(tempstring[:-2] + ', [custom ffi lib]')

        tempstring='        '
        for onepar in callParameters:
            tempstring += callParameters[onepar] + ' ' + onepar + ', '

        print(tempstring[:-2])

        print()

        print('        > callback functions:')

        for oneCallback in callbackfunctions:
            tempstring='        '  + str(oneCallback[7:])
            print(tempstring)
        print('    """')



    praefix = ''#'    '
    print(praefix + '@safeUtils.safeThread(timeout=5,queue=self.queue)')
    callbackNames = [(lambda x: '_' + nameOfFunction + '_' + x)(x) for x in callbackDict.keys()]
    funDef, innerParameters = getFunctionDefPy(nameOfFunction,CallParameters,praefix,callbackDict.keys())
    print(funDef)
    #print('')
    printMethodDefinition(CallParameters,callbackfunctions,nameOfFunction)

    praefix = praefix + '    '
    libCall = 'safenetLib.'+nameOfFunction+'('+innerParameters
    
    for callback in callbackDict:
        declaration = praefix + f'@ffi.callback("void('
        for innerparam in callbackDict[callback]:
            declaration += callbackDict[callback][innerparam] + ' ,'
        declaration = declaration[:-2] + ')")'
        print(declaration)


        definition = praefix + 'def _' + nameOfFunction + '_' + str(callback) +'('
        innerparams = ''
        for innerparam in callbackDict[callback]:
            definition += innerparam + ' ,'
            innerparams+= innerparam + ' ,'
        definition = definition[:-2] + '):'
        print(definition)
        if 'result' in definition:
            print(praefix + '    ' + 'self.safeUtils.checkResult(result)')
        print(praefix + '    ' + 'self.queue.put(\'gotResult\')')
        print(praefix + '    ' + 'if ' + callback + ':')
        print(praefix + '    ' + '    ' + callback + '(' + innerparams[:-2] + ')')

        print()
        print()

        libCall += ', _' + nameOfFunction + '_' + str(callback)

    print(praefix+libCall + ')')
    print()
    print()
    print()

@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_init_logging(self, output_file_name_override, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name_override, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_init_logging_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_init_logging(output_file_name_override, user_data, _auth_init_logging_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_output_log_path(self, output_file_name, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name, void* user_data

        > callback func

In [1]:
import safeUtils

In [2]:
import queue

In [4]:
oneQueue = queue.Queue()

In [6]:
oneQueue.empty()

True

In [7]:
@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_init_logging(self, output_file_name_override, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name_override, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_init_logging_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_init_logging(output_file_name_override, user_data, _auth_init_logging_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_output_log_path(self, output_file_name, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* log_path)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_output_log_path_o_cb(user_data ,result ,log_path):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,log_path)


    safenetLib.auth_output_log_path(output_file_name, user_data, _auth_output_log_path_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _create_acc(self, account_locator, account_password, invitation, user_data, o_disconnect_notifier_cb=None, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, bytes, bytes, [any], [function], [function], [custom ffi lib]
        char* account_locator, char* account_password, char* invitation, void* user_data

        > callback functions:
        (*o_disconnect_notifier_cb)(void* user_data)
        (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator)
    """
    @ffi.callback("void(void*)")
    def _create_acc_o_disconnect_notifier_cb(user_data):
        self.queue.put('gotResult')
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)


    @ffi.callback("void(void* ,FfiResult* ,Authenticator*)")
    def _create_acc_o_cb(user_data ,result ,authenticator):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,authenticator)


    safenetLib.create_acc(account_locator, account_password, invitation, user_data, _create_acc_o_disconnect_notifier_cb, _create_acc_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _login(self, account_locator, account_password, user_data, o_disconnect_notifier_cb=None, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, bytes, [any], [function], [function], [custom ffi lib]
        char* account_locator, char* account_password, void* user_data

        > callback functions:
        (*o_disconnect_notifier_cb)(void* user_data)
        (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticaor)
    """
    @ffi.callback("void(void*)")
    def _login_o_disconnect_notifier_cb(user_data):
        self.queue.put('gotResult')
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)


    @ffi.callback("void(void* ,FfiResult* ,Authenticator*)")
    def _login_o_cb(user_data ,result ,authenticaor):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,authenticaor)


    safenetLib.login(account_locator, account_password, user_data, _login_o_disconnect_notifier_cb, _login_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_reconnect(self, auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_reconnect_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_reconnect(auth, user_data, _auth_reconnect_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_account_info(self, auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AccountInfo* account_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,AccountInfo*)")
    def _auth_account_info_o_cb(user_data ,result ,account_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,account_info)


    safenetLib.auth_account_info(auth, user_data, _auth_account_info_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_exe_file_stem(self, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        [any], [function], [custom ffi lib]
        void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* filename)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_exe_file_stem_o_cb(user_data ,result ,filename):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,filename)


    safenetLib.auth_exe_file_stem(user_data, _auth_exe_file_stem_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_set_additional_search_path(self, new_path, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* new_path, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_set_additional_search_path_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_set_additional_search_path(new_path, user_data, _auth_set_additional_search_path_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_free(self, auth, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [custom ffi lib]
        Authenticator* auth

        > callback functions:
    """
    safenetLib.auth_free(auth)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_unregistered_decode_ipc_msg(self, msg, user_data, o_unregistered=None, o_err=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [function], [custom ffi lib]
        char* msg, void* user_data

        > callback functions:
        (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* extra_data, uintptr_t extra_data_len)
        (*o_err)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,uint32_t ,uint8_t* ,uintptr_t)")
    def _auth_unregistered_decode_ipc_msg_o_unregistered(user_data ,req_id ,extra_data ,extra_data_len):
        self.queue.put('gotResult')
        if o_unregistered:
            o_unregistered(user_data ,req_id ,extra_data ,extra_data_len)


    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_unregistered_decode_ipc_msg_o_err(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_err:
            o_err(user_data ,result ,response)


    safenetLib.auth_unregistered_decode_ipc_msg(msg, user_data, _auth_unregistered_decode_ipc_msg_o_unregistered, _auth_unregistered_decode_ipc_msg_o_err)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_decode_ipc_msg(self, auth, msg, user_data, o_auth=None, o_containers=None, o_unregistered=None, o_share_mdata=None, o_err=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, bytes, [any], [function], [function], [function], [function], [function], [custom ffi lib]
        Authenticator* auth, char* msg, void* user_data

        > callback functions:
        (*o_auth)(void* user_data, uint32_t req_id, AuthReq* req)
        (*o_containers)(void* user_data, uint32_t req_id, ContainersReq* req)
        (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* extra_data, uintptr_t extra_data_len)
        (*o_share_mdata)(void* user_data, uint32_t req_id, ShareMDataReq* req, MetadataResponse* metadata, uintptr_t metadata_len)
        (*o_err)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,uint32_t ,AuthReq*)")
    def _auth_decode_ipc_msg_o_auth(user_data ,req_id ,req):
        self.queue.put('gotResult')
        if o_auth:
            o_auth(user_data ,req_id ,req)


    @ffi.callback("void(void* ,uint32_t ,ContainersReq*)")
    def _auth_decode_ipc_msg_o_containers(user_data ,req_id ,req):
        self.queue.put('gotResult')
        if o_containers:
            o_containers(user_data ,req_id ,req)


    @ffi.callback("void(void* ,uint32_t ,uint8_t* ,uintptr_t)")
    def _auth_decode_ipc_msg_o_unregistered(user_data ,req_id ,extra_data ,extra_data_len):
        self.queue.put('gotResult')
        if o_unregistered:
            o_unregistered(user_data ,req_id ,extra_data ,extra_data_len)


    @ffi.callback("void(void* ,uint32_t ,ShareMDataReq* ,MetadataResponse* ,uintptr_t)")
    def _auth_decode_ipc_msg_o_share_mdata(user_data ,req_id ,req ,metadata ,metadata_len):
        self.queue.put('gotResult')
        if o_share_mdata:
            o_share_mdata(user_data ,req_id ,req ,metadata ,metadata_len)


    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_decode_ipc_msg_o_err(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_err:
            o_err(user_data ,result ,response)


    safenetLib.auth_decode_ipc_msg(auth, msg, user_data, _auth_decode_ipc_msg_o_auth, _auth_decode_ipc_msg_o_containers, _auth_decode_ipc_msg_o_unregistered, _auth_decode_ipc_msg_o_share_mdata, _auth_decode_ipc_msg_o_err)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_share_mdata_resp(self, auth, req, req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, ShareMDataReq*, uint32_t, _Bool, [any], [function], [custom ffi lib]
        Authenticator* auth, ShareMDataReq* req, uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_share_mdata_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_share_mdata_resp(auth, req, req_id, is_granted, user_data, _encode_share_mdata_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_revoke_app(self, auth, app_id, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, bytes, [any], [function], [custom ffi lib]
        Authenticator* auth, char* app_id, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_revoke_app_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.auth_revoke_app(auth, app_id, user_data, _auth_revoke_app_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_flush_app_revocation_queue(self, auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_flush_app_revocation_queue_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_flush_app_revocation_queue(auth, user_data, _auth_flush_app_revocation_queue_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_unregistered_resp(self, req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        uint32_t, _Bool, [any], [function], [custom ffi lib]
        uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_unregistered_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_unregistered_resp(req_id, is_granted, user_data, _encode_unregistered_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_auth_resp(self, auth, req, req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, AuthReq*, uint32_t, _Bool, [any], [function], [custom ffi lib]
        Authenticator* auth, AuthReq* req, uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_auth_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_auth_resp(auth, req, req_id, is_granted, user_data, _encode_auth_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_containers_resp(self, auth, req, req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, ContainersReq*, uint32_t, _Bool, [any], [function], [custom ffi lib]
        Authenticator* auth, ContainersReq* req, uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_containers_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_containers_resp(auth, req, req_id, is_granted, user_data, _encode_containers_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_rm_revoked_app(self, auth, app_id, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, bytes, [any], [function], [custom ffi lib]
        Authenticator* auth, char* app_id, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_rm_revoked_app_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_rm_revoked_app(auth, app_id, user_data, _auth_rm_revoked_app_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_revoked_apps(self, auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AppExchangeInfo* app_exchange_info, uintptr_t app_exchange_info_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,AppExchangeInfo* ,uintptr_t)")
    def _auth_revoked_apps_o_cb(user_data ,result ,app_exchange_info ,app_exchange_info_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,app_exchange_info ,app_exchange_info_len)


    safenetLib.auth_revoked_apps(auth, user_data, _auth_revoked_apps_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_registered_apps(self, auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, RegisteredApp* registered_app, uintptr_t registered_app_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,RegisteredApp* ,uintptr_t)")
    def _auth_registered_apps_o_cb(user_data ,result ,registered_app ,registered_app_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,registered_app ,registered_app_len)


    safenetLib.auth_registered_apps(auth, user_data, _auth_registered_apps_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _auth_apps_accessing_mutable_data(self, auth, md_name, md_type_tag, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, XorNameArray*, uint64_t, [any], [function], [custom ffi lib]
        Authenticator* auth, XorNameArray* md_name, uint64_t md_type_tag, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AppAccess* app_access, uintptr_t app_access_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,AppAccess* ,uintptr_t)")
    def _auth_apps_accessing_mutable_data_o_cb(user_data ,result ,app_access ,app_access_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,app_access ,app_access_len)


    safenetLib.auth_apps_accessing_mutable_data(auth, md_name, md_type_tag, user_data, _auth_apps_accessing_mutable_data_o_cb)

In [ ]:
_auth_init_logging()

In [8]:
print(_auth_init_logging.__doc__)


        bytes, [any], [function], [custom ffi lib]
        char* output_file_name_override, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    


In [53]:
@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_init_logging(output_file_name_override, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name_override, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_init_logging_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_init_logging(output_file_name_override, user_data, _auth_init_logging_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_output_log_path(output_file_name, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* log_path)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_output_log_path_o_cb(user_data ,result ,log_path):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,log_path)


    safenetLib.auth_output_log_path(output_file_name, user_data, _auth_output_log_path_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._create_acc(account_locator, account_password, invitation, user_data, o_disconnect_notifier_cb=None, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, bytes, bytes, [any], [function], [function], [custom ffi lib]
        char* account_locator, char* account_password, char* invitation, void* user_data

        > callback functions:
        (*o_disconnect_notifier_cb)(void* user_data)
        (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticator)
    """
    @ffi.callback("void(void*)")
    def _create_acc_o_disconnect_notifier_cb(user_data):
        self.queue.put('gotResult')
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)


    @ffi.callback("void(void* ,FfiResult* ,Authenticator*)")
    def _create_acc_o_cb(user_data ,result ,authenticator):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,authenticator)


    safenetLib.create_acc(account_locator, account_password, invitation, user_data, _create_acc_o_disconnect_notifier_cb, _create_acc_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._login(account_locator, account_password, user_data, o_disconnect_notifier_cb=None, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, bytes, [any], [function], [function], [custom ffi lib]
        char* account_locator, char* account_password, void* user_data

        > callback functions:
        (*o_disconnect_notifier_cb)(void* user_data)
        (*o_cb)(void* user_data, FfiResult* result, Authenticator* authenticaor)
    """
    @ffi.callback("void(void*)")
    def _login_o_disconnect_notifier_cb(user_data):
        self.queue.put('gotResult')
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)


    @ffi.callback("void(void* ,FfiResult* ,Authenticator*)")
    def _login_o_cb(user_data ,result ,authenticaor):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,authenticaor)


    safenetLib.login(account_locator, account_password, user_data, _login_o_disconnect_notifier_cb, _login_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_reconnect(auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_reconnect_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_reconnect(auth, user_data, _auth_reconnect_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_account_info(auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AccountInfo* account_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,AccountInfo*)")
    def _auth_account_info_o_cb(user_data ,result ,account_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,account_info)


    safenetLib.auth_account_info(auth, user_data, _auth_account_info_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_exe_file_stem(user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        [any], [function], [custom ffi lib]
        void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* filename)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_exe_file_stem_o_cb(user_data ,result ,filename):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,filename)


    safenetLib.auth_exe_file_stem(user_data, _auth_exe_file_stem_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_set_additional_search_path(new_path, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [custom ffi lib]
        char* new_path, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_set_additional_search_path_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_set_additional_search_path(new_path, user_data, _auth_set_additional_search_path_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_free(auth, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [custom ffi lib]
        Authenticator* auth

        > callback functions:
    """
    safenetLib.auth_free(auth)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_unregistered_decode_ipc_msg(msg, user_data, o_unregistered=None, o_err=None, safenetLib=self.lib.safe_authenticator):
    """
        bytes, [any], [function], [function], [custom ffi lib]
        char* msg, void* user_data

        > callback functions:
        (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* extra_data, uintptr_t extra_data_len)
        (*o_err)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,uint32_t ,uint8_t* ,uintptr_t)")
    def _auth_unregistered_decode_ipc_msg_o_unregistered(user_data ,req_id ,extra_data ,extra_data_len):
        self.queue.put('gotResult')
        if o_unregistered:
            o_unregistered(user_data ,req_id ,extra_data ,extra_data_len)


    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_unregistered_decode_ipc_msg_o_err(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_err:
            o_err(user_data ,result ,response)


    safenetLib.auth_unregistered_decode_ipc_msg(msg, user_data, _auth_unregistered_decode_ipc_msg_o_unregistered, _auth_unregistered_decode_ipc_msg_o_err)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_decode_ipc_msg(auth, msg, user_data, o_auth=None, o_containers=None, o_unregistered=None, o_share_mdata=None, o_err=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, bytes, [any], [function], [function], [function], [function], [function], [custom ffi lib]
        Authenticator* auth, char* msg, void* user_data

        > callback functions:
        (*o_auth)(void* user_data, uint32_t req_id, AuthReq* req)
        (*o_containers)(void* user_data, uint32_t req_id, ContainersReq* req)
        (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* extra_data, uintptr_t extra_data_len)
        (*o_share_mdata)(void* user_data, uint32_t req_id, ShareMDataReq* req, MetadataResponse* metadata, uintptr_t metadata_len)
        (*o_err)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,uint32_t ,AuthReq*)")
    def _auth_decode_ipc_msg_o_auth(user_data ,req_id ,req):
        self.queue.put('gotResult')
        if o_auth:
            o_auth(user_data ,req_id ,req)


    @ffi.callback("void(void* ,uint32_t ,ContainersReq*)")
    def _auth_decode_ipc_msg_o_containers(user_data ,req_id ,req):
        self.queue.put('gotResult')
        if o_containers:
            o_containers(user_data ,req_id ,req)


    @ffi.callback("void(void* ,uint32_t ,uint8_t* ,uintptr_t)")
    def _auth_decode_ipc_msg_o_unregistered(user_data ,req_id ,extra_data ,extra_data_len):
        self.queue.put('gotResult')
        if o_unregistered:
            o_unregistered(user_data ,req_id ,extra_data ,extra_data_len)


    @ffi.callback("void(void* ,uint32_t ,ShareMDataReq* ,MetadataResponse* ,uintptr_t)")
    def _auth_decode_ipc_msg_o_share_mdata(user_data ,req_id ,req ,metadata ,metadata_len):
        self.queue.put('gotResult')
        if o_share_mdata:
            o_share_mdata(user_data ,req_id ,req ,metadata ,metadata_len)


    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_decode_ipc_msg_o_err(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_err:
            o_err(user_data ,result ,response)


    safenetLib.auth_decode_ipc_msg(auth, msg, user_data, _auth_decode_ipc_msg_o_auth, _auth_decode_ipc_msg_o_containers, _auth_decode_ipc_msg_o_unregistered, _auth_decode_ipc_msg_o_share_mdata, _auth_decode_ipc_msg_o_err)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._encode_share_mdata_resp(auth, req, req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, ShareMDataReq*, uint32_t, _Bool, [any], [function], [custom ffi lib]
        Authenticator* auth, ShareMDataReq* req, uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_share_mdata_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_share_mdata_resp(auth, req, req_id, is_granted, user_data, _encode_share_mdata_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_revoke_app(auth, app_id, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, bytes, [any], [function], [custom ffi lib]
        Authenticator* auth, char* app_id, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _auth_revoke_app_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.auth_revoke_app(auth, app_id, user_data, _auth_revoke_app_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_flush_app_revocation_queue(auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_flush_app_revocation_queue_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_flush_app_revocation_queue(auth, user_data, _auth_flush_app_revocation_queue_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._encode_unregistered_resp(req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        uint32_t, _Bool, [any], [function], [custom ffi lib]
        uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_unregistered_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_unregistered_resp(req_id, is_granted, user_data, _encode_unregistered_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._encode_auth_resp(auth, req, req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, AuthReq*, uint32_t, _Bool, [any], [function], [custom ffi lib]
        Authenticator* auth, AuthReq* req, uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_auth_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_auth_resp(auth, req, req_id, is_granted, user_data, _encode_auth_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._encode_containers_resp(auth, req, req_id, is_granted, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, ContainersReq*, uint32_t, _Bool, [any], [function], [custom ffi lib]
        Authenticator* auth, ContainersReq* req, uint32_t req_id, _Bool is_granted, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* response)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _encode_containers_resp_o_cb(user_data ,result ,response):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,response)


    safenetLib.encode_containers_resp(auth, req, req_id, is_granted, user_data, _encode_containers_resp_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_rm_revoked_app(auth, app_id, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, bytes, [any], [function], [custom ffi lib]
        Authenticator* auth, char* app_id, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _auth_rm_revoked_app_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.auth_rm_revoked_app(auth, app_id, user_data, _auth_rm_revoked_app_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_revoked_apps(auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AppExchangeInfo* app_exchange_info, uintptr_t app_exchange_info_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,AppExchangeInfo* ,uintptr_t)")
    def _auth_revoked_apps_o_cb(user_data ,result ,app_exchange_info ,app_exchange_info_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,app_exchange_info ,app_exchange_info_len)


    safenetLib.auth_revoked_apps(auth, user_data, _auth_revoked_apps_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_registered_apps(auth, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, [any], [function], [custom ffi lib]
        Authenticator* auth, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, RegisteredApp* registered_app, uintptr_t registered_app_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,RegisteredApp* ,uintptr_t)")
    def _auth_registered_apps_o_cb(user_data ,result ,registered_app ,registered_app_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,registered_app ,registered_app_len)


    safenetLib.auth_registered_apps(auth, user_data, _auth_registered_apps_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def self._auth_apps_accessing_mutable_data(auth, md_name, md_type_tag, user_data, o_cb=None, safenetLib=self.lib.safe_authenticator):
    """
        Authenticator*, XorNameArray*, uint64_t, [any], [function], [custom ffi lib]
        Authenticator* auth, XorNameArray* md_name, uint64_t md_type_tag, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AppAccess* app_access, uintptr_t app_access_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,AppAccess* ,uintptr_t)")
    def _auth_apps_accessing_mutable_data_o_cb(user_data ,result ,app_access ,app_access_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,app_access ,app_access_len)


    safenetLib.auth_apps_accessing_mutable_data(auth, md_name, md_type_tag, user_data, _auth_apps_accessing_mutable_data_o_cb)


SyntaxError: invalid syntax (<ipython-input-53-23a9abc371e9>, line 2)

In [ ]:
@safeUtils.safeThread(timeout=5,queue=self.queue)
def _test_create_app(self, app_id, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        bytes, [any], [function], [custom ffi lib]
        char* app_id, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, App* app)
    """
    @ffi.callback("void(void* ,FfiResult* ,App*)")
    def _test_create_app_o_cb(user_data ,result ,app):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,app)


    safenetLib.test_create_app(app_id, user_data, _test_create_app_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _test_create_app_with_access(self, auth_req, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        AuthReq*, [any], [function], [custom ffi lib]
        AuthReq* auth_req, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, App* o_app)
    """
    @ffi.callback("void(void* ,FfiResult* ,App*)")
    def _test_create_app_with_access_o_cb(user_data ,result ,o_app):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,o_app)


    safenetLib.test_create_app_with_access(auth_req, user_data, _test_create_app_with_access_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _test_simulate_network_disconnect(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _test_simulate_network_disconnect_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.test_simulate_network_disconnect(app, user_data, _test_simulate_network_disconnect_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_init_logging(self, output_file_name_override, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name_override, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _app_init_logging_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.app_init_logging(output_file_name_override, user_data, _app_init_logging_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_output_log_path(self, output_file_name, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        bytes, [any], [function], [custom ffi lib]
        char* output_file_name, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* log_path)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _app_output_log_path_o_cb(user_data ,result ,log_path):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,log_path)


    safenetLib.app_output_log_path(output_file_name, user_data, _app_output_log_path_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_encode_metadata(self, metadata, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        MetadataResponse*, [any], [function], [custom ffi lib]
        MetadataResponse* metadata, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* encoded, uintptr_t encoded_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _mdata_encode_metadata_o_cb(user_data ,result ,encoded ,encoded_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,encoded ,encoded_len)


    safenetLib.mdata_encode_metadata(metadata, user_data, _mdata_encode_metadata_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_new_private(self, name, type_tag, secret_key, nonce, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        XorNameArray*, uint64_t, SymSecretKey*, SymNonce*, [any], [function], [custom ffi lib]
        XorNameArray* name, uint64_t type_tag, SymSecretKey* secret_key, SymNonce* nonce, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataInfo* mdata_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataInfo*)")
    def _mdata_info_new_private_o_cb(user_data ,result ,mdata_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,mdata_info)


    safenetLib.mdata_info_new_private(name, type_tag, secret_key, nonce, user_data, _mdata_info_new_private_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_random_public(self, type_tag, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        uint64_t, [any], [function], [custom ffi lib]
        uint64_t type_tag, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataInfo* mdata_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataInfo*)")
    def _mdata_info_random_public_o_cb(user_data ,result ,mdata_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,mdata_info)


    safenetLib.mdata_info_random_public(type_tag, user_data, _mdata_info_random_public_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_random_private(self, type_tag, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        uint64_t, [any], [function], [custom ffi lib]
        uint64_t type_tag, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataInfo* mdata_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataInfo*)")
    def _mdata_info_random_private_o_cb(user_data ,result ,mdata_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,mdata_info)


    safenetLib.mdata_info_random_private(type_tag, user_data, _mdata_info_random_private_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_encrypt_entry_key(self, info, input, input_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        MDataInfo*, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        MDataInfo* info, uint8_t* input, uintptr_t input_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* enc_entry_key, uintptr_t enc_entry_key_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _mdata_info_encrypt_entry_key_o_cb(user_data ,result ,enc_entry_key ,enc_entry_key_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,enc_entry_key ,enc_entry_key_len)


    safenetLib.mdata_info_encrypt_entry_key(info, input, input_len, user_data, _mdata_info_encrypt_entry_key_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_encrypt_entry_value(self, info, input, input_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        MDataInfo*, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        MDataInfo* info, uint8_t* input, uintptr_t input_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* enc_entry_value, uintptr_t enc_entry_value_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _mdata_info_encrypt_entry_value_o_cb(user_data ,result ,enc_entry_value ,enc_entry_value_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,enc_entry_value ,enc_entry_value_len)


    safenetLib.mdata_info_encrypt_entry_value(info, input, input_len, user_data, _mdata_info_encrypt_entry_value_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_decrypt(self, info, input, input_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        MDataInfo*, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        MDataInfo* info, uint8_t* input, uintptr_t input_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* mdata_info_decrypt, uintptr_t mdata_info_decrypt_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _mdata_info_decrypt_o_cb(user_data ,result ,mdata_info_decrypt ,mdata_info_decrypt_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,mdata_info_decrypt ,mdata_info_decrypt_len)


    safenetLib.mdata_info_decrypt(info, input, input_len, user_data, _mdata_info_decrypt_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_serialise(self, info, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        MDataInfo*, [any], [function], [custom ffi lib]
        MDataInfo* info, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* encoded, uintptr_t encoded_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _mdata_info_serialise_o_cb(user_data ,result ,encoded ,encoded_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,encoded ,encoded_len)


    safenetLib.mdata_info_serialise(info, user_data, _mdata_info_serialise_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_info_deserialise(self, encoded_ptr, encoded_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        uint8_t* encoded_ptr, uintptr_t encoded_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataInfo* mdata_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataInfo*)")
    def _mdata_info_deserialise_o_cb(user_data ,result ,mdata_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,mdata_info)


    safenetLib.mdata_info_deserialise(encoded_ptr, encoded_len, user_data, _mdata_info_deserialise_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_unregistered(self, bootstrap_config, bootstrap_config_len, user_data, o_disconnect_notifier_cb=None, o_cb=None, safenetLib=self.lib.safe_app):
    """
        uint8_t*, uintptr_t, [any], [function], [function], [custom ffi lib]
        uint8_t* bootstrap_config, uintptr_t bootstrap_config_len, void* user_data

        > callback functions:
        (*o_disconnect_notifier_cb)(void* user_data)
        (*o_cb)(void* user_data, FfiResult* result, App* app)
    """
    @ffi.callback("void(void*)")
    def _app_unregistered_o_disconnect_notifier_cb(user_data):
        self.queue.put('gotResult')
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)


    @ffi.callback("void(void* ,FfiResult* ,App*)")
    def _app_unregistered_o_cb(user_data ,result ,app):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,app)


    safenetLib.app_unregistered(bootstrap_config, bootstrap_config_len, user_data, _app_unregistered_o_disconnect_notifier_cb, _app_unregistered_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_registered(self, app_id, auth_granted, user_data, o_disconnect_notifier_cb=None, o_cb=None, safenetLib=self.lib.safe_app):
    """
        bytes, AuthGranted*, [any], [function], [function], [custom ffi lib]
        char* app_id, AuthGranted* auth_granted, void* user_data

        > callback functions:
        (*o_disconnect_notifier_cb)(void* user_data)
        (*o_cb)(void* user_data, FfiResult* result, App* app)
    """
    @ffi.callback("void(void*)")
    def _app_registered_o_disconnect_notifier_cb(user_data):
        self.queue.put('gotResult')
        if o_disconnect_notifier_cb:
            o_disconnect_notifier_cb(user_data)


    @ffi.callback("void(void* ,FfiResult* ,App*)")
    def _app_registered_o_cb(user_data ,result ,app):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,app)


    safenetLib.app_registered(app_id, auth_granted, user_data, _app_registered_o_disconnect_notifier_cb, _app_registered_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_reconnect(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _app_reconnect_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.app_reconnect(app, user_data, _app_reconnect_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_account_info(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AccountInfo* account_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,AccountInfo*)")
    def _app_account_info_o_cb(user_data ,result ,account_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,account_info)


    safenetLib.app_account_info(app, user_data, _app_account_info_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_exe_file_stem(self, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        [any], [function], [custom ffi lib]
        void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* filename)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _app_exe_file_stem_o_cb(user_data ,result ,filename):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,filename)


    safenetLib.app_exe_file_stem(user_data, _app_exe_file_stem_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_set_additional_search_path(self, new_path, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        bytes, [any], [function], [custom ffi lib]
        char* new_path, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _app_set_additional_search_path_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.app_set_additional_search_path(new_path, user_data, _app_set_additional_search_path_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_free(self, app, safenetLib=self.lib.safe_app):
    """
        App*, [custom ffi lib]
        App* app

        > callback functions:
    """
    safenetLib.app_free(app)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_reset_object_cache(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _app_reset_object_cache_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.app_reset_object_cache(app, user_data, _app_reset_object_cache_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_container_name(self, app_id, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        bytes, [any], [function], [custom ffi lib]
        char* app_id, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, char* container_name)
    """
    @ffi.callback("void(void* ,FfiResult* ,char*)")
    def _app_container_name_o_cb(user_data ,result ,container_name):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,container_name)


    safenetLib.app_container_name(app_id, user_data, _app_container_name_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_auth_req(self, req, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        AuthReq*, [any], [function], [custom ffi lib]
        AuthReq* req, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint32_t req_id, char* encoded)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint32_t ,char*)")
    def _encode_auth_req_o_cb(user_data ,result ,req_id ,encoded):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,req_id ,encoded)


    safenetLib.encode_auth_req(req, user_data, _encode_auth_req_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_containers_req(self, req, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        ContainersReq*, [any], [function], [custom ffi lib]
        ContainersReq* req, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint32_t req_id, char* encoded)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint32_t ,char*)")
    def _encode_containers_req_o_cb(user_data ,result ,req_id ,encoded):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,req_id ,encoded)


    safenetLib.encode_containers_req(req, user_data, _encode_containers_req_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_unregistered_req(self, extra_data, extra_data_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        uint8_t* extra_data, uintptr_t extra_data_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint32_t req_id, char* encoded)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint32_t ,char*)")
    def _encode_unregistered_req_o_cb(user_data ,result ,req_id ,encoded):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,req_id ,encoded)


    safenetLib.encode_unregistered_req(extra_data, extra_data_len, user_data, _encode_unregistered_req_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encode_share_mdata_req(self, req, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        ShareMDataReq*, [any], [function], [custom ffi lib]
        ShareMDataReq* req, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint32_t req_id, char* encoded)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint32_t ,char*)")
    def _encode_share_mdata_req_o_cb(user_data ,result ,req_id ,encoded):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,req_id ,encoded)


    safenetLib.encode_share_mdata_req(req, user_data, _encode_share_mdata_req_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _decode_ipc_msg(self, msg, user_data, o_auth=None, o_unregistered=None, o_containers=None, o_share_mdata=None, o_revoked=None, o_err=None, safenetLib=self.lib.safe_app):
    """
        bytes, [any], [function], [function], [function], [function], [function], [function], [custom ffi lib]
        char* msg, void* user_data

        > callback functions:
        (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted)
        (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, uintptr_t serialised_cfg_len)
        (*o_containers)(void* user_data, uint32_t req_id)
        (*o_share_mdata)(void* user_data, uint32_t req_id)
        (*o_revoked)(void* user_data)
        (*o_err)(void* user_data, FfiResult* result, uint32_t req_id)
    """
    @ffi.callback("void(void* ,uint32_t ,AuthGranted*)")
    def _decode_ipc_msg_o_auth(user_data ,req_id ,auth_granted):
        self.queue.put('gotResult')
        if o_auth:
            o_auth(user_data ,req_id ,auth_granted)


    @ffi.callback("void(void* ,uint32_t ,uint8_t* ,uintptr_t)")
    def _decode_ipc_msg_o_unregistered(user_data ,req_id ,serialised_cfg ,serialised_cfg_len):
        self.queue.put('gotResult')
        if o_unregistered:
            o_unregistered(user_data ,req_id ,serialised_cfg ,serialised_cfg_len)


    @ffi.callback("void(void* ,uint32_t)")
    def _decode_ipc_msg_o_containers(user_data ,req_id):
        self.queue.put('gotResult')
        if o_containers:
            o_containers(user_data ,req_id)


    @ffi.callback("void(void* ,uint32_t)")
    def _decode_ipc_msg_o_share_mdata(user_data ,req_id):
        self.queue.put('gotResult')
        if o_share_mdata:
            o_share_mdata(user_data ,req_id)


    @ffi.callback("void(void*)")
    def _decode_ipc_msg_o_revoked(user_data):
        self.queue.put('gotResult')
        if o_revoked:
            o_revoked(user_data)


    @ffi.callback("void(void* ,FfiResult* ,uint32_t)")
    def _decode_ipc_msg_o_err(user_data ,result ,req_id):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_err:
            o_err(user_data ,result ,req_id)


    safenetLib.decode_ipc_msg(msg, user_data, _decode_ipc_msg_o_auth, _decode_ipc_msg_o_unregistered, _decode_ipc_msg_o_containers, _decode_ipc_msg_o_share_mdata, _decode_ipc_msg_o_revoked, _decode_ipc_msg_o_err)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _access_container_refresh_access_info(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _access_container_refresh_access_info_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.access_container_refresh_access_info(app, user_data, _access_container_refresh_access_info_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _access_container_fetch(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, ContainerPermissions* container_perms, uintptr_t container_perms_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,ContainerPermissions* ,uintptr_t)")
    def _access_container_fetch_o_cb(user_data ,result ,container_perms ,container_perms_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,container_perms ,container_perms_len)


    safenetLib.access_container_fetch(app, user_data, _access_container_fetch_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _access_container_get_container_mdata_info(self, app, name, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, bytes, [any], [function], [custom ffi lib]
        App* app, char* name, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataInfo* mdata_info)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataInfo*)")
    def _access_container_get_container_mdata_info_o_cb(user_data ,result ,mdata_info):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,mdata_info)


    safenetLib.access_container_get_container_mdata_info(app, name, user_data, _access_container_get_container_mdata_info_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_put(self, app, info, permissions_h, entries_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, MDataPermissionsHandle, MDataEntriesHandle, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, MDataPermissionsHandle permissions_h, MDataEntriesHandle entries_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_put_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_put(app, info, permissions_h, entries_h, user_data, _mdata_put_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_get_version(self, app, info, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint64_t version)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint64_t)")
    def _mdata_get_version_o_cb(user_data ,result ,version):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,version)


    safenetLib.mdata_get_version(app, info, user_data, _mdata_get_version_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_serialised_size(self, app, info, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint64_t serialised_size)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint64_t)")
    def _mdata_serialised_size_o_cb(user_data ,result ,serialised_size):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,serialised_size)


    safenetLib.mdata_serialised_size(app, info, user_data, _mdata_serialised_size_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_get_value(self, app, info, key, key_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, uint8_t* key, uintptr_t key_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* content, uintptr_t content_len, uint64_t version)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t ,uint64_t)")
    def _mdata_get_value_o_cb(user_data ,result ,content ,content_len ,version):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,content ,content_len ,version)


    safenetLib.mdata_get_value(app, info, key, key_len, user_data, _mdata_get_value_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entries(self, app, info, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataEntriesHandle entries_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataEntriesHandle)")
    def _mdata_entries_o_cb(user_data ,result ,entries_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,entries_h)


    safenetLib.mdata_entries(app, info, user_data, _mdata_entries_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_list_keys(self, app, info, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataKey* keys, uintptr_t keys_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataKey* ,uintptr_t)")
    def _mdata_list_keys_o_cb(user_data ,result ,keys ,keys_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,keys ,keys_len)


    safenetLib.mdata_list_keys(app, info, user_data, _mdata_list_keys_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_list_values(self, app, info, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataValue* values, uintptr_t values_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataValue* ,uintptr_t)")
    def _mdata_list_values_o_cb(user_data ,result ,values ,values_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,values ,values_len)


    safenetLib.mdata_list_values(app, info, user_data, _mdata_list_values_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_mutate_entries(self, app, info, actions_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, MDataEntryActionsHandle, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, MDataEntryActionsHandle actions_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_mutate_entries_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_mutate_entries(app, info, actions_h, user_data, _mdata_mutate_entries_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_list_permissions(self, app, info, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataPermissionsHandle perm_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataPermissionsHandle)")
    def _mdata_list_permissions_o_cb(user_data ,result ,perm_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,perm_h)


    safenetLib.mdata_list_permissions(app, info, user_data, _mdata_list_permissions_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_list_user_permissions(self, app, info, user_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, SignPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, SignPubKeyHandle user_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, PermissionSet* perm_set)
    """
    @ffi.callback("void(void* ,FfiResult* ,PermissionSet*)")
    def _mdata_list_user_permissions_o_cb(user_data ,result ,perm_set):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,perm_set)


    safenetLib.mdata_list_user_permissions(app, info, user_h, user_data, _mdata_list_user_permissions_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_set_user_permissions(self, app, info, user_h, permission_set, version, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, SignPubKeyHandle, PermissionSet*, uint64_t, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, SignPubKeyHandle user_h, PermissionSet* permission_set, uint64_t version, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_set_user_permissions_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_set_user_permissions(app, info, user_h, permission_set, version, user_data, _mdata_set_user_permissions_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_del_user_permissions(self, app, info, user_h, version, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, SignPubKeyHandle, uint64_t, [any], [function], [custom ffi lib]
        App* app, MDataInfo* info, SignPubKeyHandle user_h, uint64_t version, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_del_user_permissions_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_del_user_permissions(app, info, user_h, version, user_data, _mdata_del_user_permissions_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_permissions_new(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataPermissionsHandle perm_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataPermissionsHandle)")
    def _mdata_permissions_new_o_cb(user_data ,result ,perm_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,perm_h)


    safenetLib.mdata_permissions_new(app, user_data, _mdata_permissions_new_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_permissions_len(self, app, permissions_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataPermissionsHandle, [any], [function], [custom ffi lib]
        App* app, MDataPermissionsHandle permissions_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uintptr_t size)
    """
    @ffi.callback("void(void* ,FfiResult* ,uintptr_t)")
    def _mdata_permissions_len_o_cb(user_data ,result ,size):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,size)


    safenetLib.mdata_permissions_len(app, permissions_h, user_data, _mdata_permissions_len_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_permissions_get(self, app, permissions_h, user_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataPermissionsHandle, SignPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, MDataPermissionsHandle permissions_h, SignPubKeyHandle user_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, PermissionSet* perm_set)
    """
    @ffi.callback("void(void* ,FfiResult* ,PermissionSet*)")
    def _mdata_permissions_get_o_cb(user_data ,result ,perm_set):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,perm_set)


    safenetLib.mdata_permissions_get(app, permissions_h, user_h, user_data, _mdata_permissions_get_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_list_permission_sets(self, app, permissions_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataPermissionsHandle, [any], [function], [custom ffi lib]
        App* app, MDataPermissionsHandle permissions_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, UserPermissionSet* user_perm_sets, uintptr_t user_perm_sets_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,UserPermissionSet* ,uintptr_t)")
    def _mdata_list_permission_sets_o_cb(user_data ,result ,user_perm_sets ,user_perm_sets_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,user_perm_sets ,user_perm_sets_len)


    safenetLib.mdata_list_permission_sets(app, permissions_h, user_data, _mdata_list_permission_sets_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_permissions_insert(self, app, permissions_h, user_h, permission_set, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataPermissionsHandle, SignPubKeyHandle, PermissionSet*, [any], [function], [custom ffi lib]
        App* app, MDataPermissionsHandle permissions_h, SignPubKeyHandle user_h, PermissionSet* permission_set, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_permissions_insert_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_permissions_insert(app, permissions_h, user_h, permission_set, user_data, _mdata_permissions_insert_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_permissions_free(self, app, permissions_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataPermissionsHandle, [any], [function], [custom ffi lib]
        App* app, MDataPermissionsHandle permissions_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_permissions_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_permissions_free(app, permissions_h, user_data, _mdata_permissions_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entry_actions_new(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataEntryActionsHandle entry_actions_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataEntryActionsHandle)")
    def _mdata_entry_actions_new_o_cb(user_data ,result ,entry_actions_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,entry_actions_h)


    safenetLib.mdata_entry_actions_new(app, user_data, _mdata_entry_actions_new_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entry_actions_insert(self, app, actions_h, key, key_len, value, value_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntryActionsHandle, uint8_t*, uintptr_t, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        App* app, MDataEntryActionsHandle actions_h, uint8_t* key, uintptr_t key_len, uint8_t* value, uintptr_t value_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_entry_actions_insert_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_entry_actions_insert(app, actions_h, key, key_len, value, value_len, user_data, _mdata_entry_actions_insert_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entry_actions_update(self, app, actions_h, key, key_len, value, value_len, entry_version, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntryActionsHandle, uint8_t*, uintptr_t, uint8_t*, uintptr_t, uint64_t, [any], [function], [custom ffi lib]
        App* app, MDataEntryActionsHandle actions_h, uint8_t* key, uintptr_t key_len, uint8_t* value, uintptr_t value_len, uint64_t entry_version, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_entry_actions_update_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_entry_actions_update(app, actions_h, key, key_len, value, value_len, entry_version, user_data, _mdata_entry_actions_update_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entry_actions_delete(self, app, actions_h, key, key_len, entry_version, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntryActionsHandle, uint8_t*, uintptr_t, uint64_t, [any], [function], [custom ffi lib]
        App* app, MDataEntryActionsHandle actions_h, uint8_t* key, uintptr_t key_len, uint64_t entry_version, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_entry_actions_delete_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_entry_actions_delete(app, actions_h, key, key_len, entry_version, user_data, _mdata_entry_actions_delete_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entry_actions_free(self, app, actions_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntryActionsHandle, [any], [function], [custom ffi lib]
        App* app, MDataEntryActionsHandle actions_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_entry_actions_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_entry_actions_free(app, actions_h, user_data, _mdata_entry_actions_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _dir_fetch_file(self, app, parent_info, file_name, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, bytes, [any], [function], [custom ffi lib]
        App* app, MDataInfo* parent_info, char* file_name, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, File* file, uint64_t version)
    """
    @ffi.callback("void(void* ,FfiResult* ,File* ,uint64_t)")
    def _dir_fetch_file_o_cb(user_data ,result ,file ,version):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,file ,version)


    safenetLib.dir_fetch_file(app, parent_info, file_name, user_data, _dir_fetch_file_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _dir_insert_file(self, app, parent_info, file_name, file, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, bytes, File*, [any], [function], [custom ffi lib]
        App* app, MDataInfo* parent_info, char* file_name, File* file, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _dir_insert_file_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.dir_insert_file(app, parent_info, file_name, file, user_data, _dir_insert_file_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _dir_update_file(self, app, parent_info, file_name, file, version, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, bytes, File*, uint64_t, [any], [function], [custom ffi lib]
        App* app, MDataInfo* parent_info, char* file_name, File* file, uint64_t version, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint64_t new_version)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint64_t)")
    def _dir_update_file_o_cb(user_data ,result ,new_version):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,new_version)


    safenetLib.dir_update_file(app, parent_info, file_name, file, version, user_data, _dir_update_file_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _dir_delete_file(self, app, parent_info, file_name, version, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, bytes, uint64_t, [any], [function], [custom ffi lib]
        App* app, MDataInfo* parent_info, char* file_name, uint64_t version, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint64_t new_version)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint64_t)")
    def _dir_delete_file_o_cb(user_data ,result ,new_version):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,new_version)


    safenetLib.dir_delete_file(app, parent_info, file_name, version, user_data, _dir_delete_file_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _file_open(self, app, parent_info, file, open_mode, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataInfo*, File*, uint64_t, [any], [function], [custom ffi lib]
        App* app, MDataInfo* parent_info, File* file, uint64_t open_mode, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, FileContextHandle file_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,FileContextHandle)")
    def _file_open_o_cb(user_data ,result ,file_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,file_h)


    safenetLib.file_open(app, parent_info, file, open_mode, user_data, _file_open_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _file_size(self, app, file_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, FileContextHandle, [any], [function], [custom ffi lib]
        App* app, FileContextHandle file_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint64_t size)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint64_t)")
    def _file_size_o_cb(user_data ,result ,size):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,size)


    safenetLib.file_size(app, file_h, user_data, _file_size_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _file_read(self, app, file_h, position, len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, FileContextHandle, uint64_t, uint64_t, [any], [function], [custom ffi lib]
        App* app, FileContextHandle file_h, uint64_t position, uint64_t len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* data, uintptr_t data_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _file_read_o_cb(user_data ,result ,data ,data_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,data ,data_len)


    safenetLib.file_read(app, file_h, position, len, user_data, _file_read_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _file_write(self, app, file_h, data, data_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, FileContextHandle, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        App* app, FileContextHandle file_h, uint8_t* data, uintptr_t data_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _file_write_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.file_write(app, file_h, data, data_len, user_data, _file_write_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _file_close(self, app, file_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, FileContextHandle, [any], [function], [custom ffi lib]
        App* app, FileContextHandle file_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, File* file)
    """
    @ffi.callback("void(void* ,FfiResult* ,File*)")
    def _file_close_o_cb(user_data ,result ,file):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,file)


    safenetLib.file_close(app, file_h, user_data, _file_close_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_new_self_encryptor(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SEWriterHandle se_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,SEWriterHandle)")
    def _idata_new_self_encryptor_o_cb(user_data ,result ,se_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,se_h)


    safenetLib.idata_new_self_encryptor(app, user_data, _idata_new_self_encryptor_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_write_to_self_encryptor(self, app, se_h, data, data_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SEWriterHandle, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        App* app, SEWriterHandle se_h, uint8_t* data, uintptr_t data_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _idata_write_to_self_encryptor_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.idata_write_to_self_encryptor(app, se_h, data, data_len, user_data, _idata_write_to_self_encryptor_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_close_self_encryptor(self, app, se_h, cipher_opt_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SEWriterHandle, CipherOptHandle, [any], [function], [custom ffi lib]
        App* app, SEWriterHandle se_h, CipherOptHandle cipher_opt_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, XorNameArray* name)
    """
    @ffi.callback("void(void* ,FfiResult* ,XorNameArray*)")
    def _idata_close_self_encryptor_o_cb(user_data ,result ,name):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,name)


    safenetLib.idata_close_self_encryptor(app, se_h, cipher_opt_h, user_data, _idata_close_self_encryptor_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_fetch_self_encryptor(self, app, name, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, XorNameArray*, [any], [function], [custom ffi lib]
        App* app, XorNameArray* name, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SEReaderHandle se_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,SEReaderHandle)")
    def _idata_fetch_self_encryptor_o_cb(user_data ,result ,se_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,se_h)


    safenetLib.idata_fetch_self_encryptor(app, name, user_data, _idata_fetch_self_encryptor_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_serialised_size(self, app, name, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, XorNameArray*, [any], [function], [custom ffi lib]
        App* app, XorNameArray* name, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint64_t serialised_size)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint64_t)")
    def _idata_serialised_size_o_cb(user_data ,result ,serialised_size):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,serialised_size)


    safenetLib.idata_serialised_size(app, name, user_data, _idata_serialised_size_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_size(self, app, se_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SEReaderHandle, [any], [function], [custom ffi lib]
        App* app, SEReaderHandle se_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint64_t size)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint64_t)")
    def _idata_size_o_cb(user_data ,result ,size):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,size)


    safenetLib.idata_size(app, se_h, user_data, _idata_size_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_read_from_self_encryptor(self, app, se_h, from_pos, len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SEReaderHandle, uint64_t, uint64_t, [any], [function], [custom ffi lib]
        App* app, SEReaderHandle se_h, uint64_t from_pos, uint64_t len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* data, uintptr_t data_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _idata_read_from_self_encryptor_o_cb(user_data ,result ,data ,data_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,data ,data_len)


    safenetLib.idata_read_from_self_encryptor(app, se_h, from_pos, len, user_data, _idata_read_from_self_encryptor_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_self_encryptor_writer_free(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SEWriterHandle, [any], [function], [custom ffi lib]
        App* app, SEWriterHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _idata_self_encryptor_writer_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.idata_self_encryptor_writer_free(app, handle, user_data, _idata_self_encryptor_writer_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _idata_self_encryptor_reader_free(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SEReaderHandle, [any], [function], [custom ffi lib]
        App* app, SEReaderHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _idata_self_encryptor_reader_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.idata_self_encryptor_reader_free(app, handle, user_data, _idata_self_encryptor_reader_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_pub_sign_key(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SignPubKeyHandle handle)
    """
    @ffi.callback("void(void* ,FfiResult* ,SignPubKeyHandle)")
    def _app_pub_sign_key_o_cb(user_data ,result ,handle):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,handle)


    safenetLib.app_pub_sign_key(app, user_data, _app_pub_sign_key_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign_generate_key_pair(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SignPubKeyHandle public_key_h, SignSecKeyHandle secret_key_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,SignPubKeyHandle ,SignSecKeyHandle)")
    def _sign_generate_key_pair_o_cb(user_data ,result ,public_key_h ,secret_key_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,public_key_h ,secret_key_h)


    safenetLib.sign_generate_key_pair(app, user_data, _sign_generate_key_pair_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign_pub_key_new(self, app, data, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SignPublicKey*, [any], [function], [custom ffi lib]
        App* app, SignPublicKey* data, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SignPubKeyHandle handle)
    """
    @ffi.callback("void(void* ,FfiResult* ,SignPubKeyHandle)")
    def _sign_pub_key_new_o_cb(user_data ,result ,handle):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,handle)


    safenetLib.sign_pub_key_new(app, data, user_data, _sign_pub_key_new_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign_pub_key_get(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SignPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, SignPubKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SignPublicKey* pub_sign_key)
    """
    @ffi.callback("void(void* ,FfiResult* ,SignPublicKey*)")
    def _sign_pub_key_get_o_cb(user_data ,result ,pub_sign_key):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,pub_sign_key)


    safenetLib.sign_pub_key_get(app, handle, user_data, _sign_pub_key_get_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign_pub_key_free(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SignPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, SignPubKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _sign_pub_key_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.sign_pub_key_free(app, handle, user_data, _sign_pub_key_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign_sec_key_new(self, app, data, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SignSecretKey*, [any], [function], [custom ffi lib]
        App* app, SignSecretKey* data, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SignSecKeyHandle handle)
    """
    @ffi.callback("void(void* ,FfiResult* ,SignSecKeyHandle)")
    def _sign_sec_key_new_o_cb(user_data ,result ,handle):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,handle)


    safenetLib.sign_sec_key_new(app, data, user_data, _sign_sec_key_new_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign_sec_key_get(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SignSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, SignSecKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, SignSecretKey* pub_sign_key)
    """
    @ffi.callback("void(void* ,FfiResult* ,SignSecretKey*)")
    def _sign_sec_key_get_o_cb(user_data ,result ,pub_sign_key):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,pub_sign_key)


    safenetLib.sign_sec_key_get(app, handle, user_data, _sign_sec_key_get_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign_sec_key_free(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, SignSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, SignSecKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _sign_sec_key_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.sign_sec_key_free(app, handle, user_data, _sign_sec_key_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _app_pub_enc_key(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, EncryptPubKeyHandle public_key_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,EncryptPubKeyHandle)")
    def _app_pub_enc_key_o_cb(user_data ,result ,public_key_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,public_key_h)


    safenetLib.app_pub_enc_key(app, user_data, _app_pub_enc_key_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _enc_generate_key_pair(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, EncryptPubKeyHandle public_key_h, EncryptSecKeyHandle secret_key_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,EncryptPubKeyHandle ,EncryptSecKeyHandle)")
    def _enc_generate_key_pair_o_cb(user_data ,result ,public_key_h ,secret_key_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,public_key_h ,secret_key_h)


    safenetLib.enc_generate_key_pair(app, user_data, _enc_generate_key_pair_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _enc_pub_key_new(self, app, data, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, AsymPublicKey*, [any], [function], [custom ffi lib]
        App* app, AsymPublicKey* data, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, EncryptPubKeyHandle public_key_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,EncryptPubKeyHandle)")
    def _enc_pub_key_new_o_cb(user_data ,result ,public_key_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,public_key_h)


    safenetLib.enc_pub_key_new(app, data, user_data, _enc_pub_key_new_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _enc_pub_key_get(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, EncryptPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, EncryptPubKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AsymPublicKey* pub_enc_key)
    """
    @ffi.callback("void(void* ,FfiResult* ,AsymPublicKey*)")
    def _enc_pub_key_get_o_cb(user_data ,result ,pub_enc_key):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,pub_enc_key)


    safenetLib.enc_pub_key_get(app, handle, user_data, _enc_pub_key_get_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _enc_pub_key_free(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, EncryptPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, EncryptPubKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _enc_pub_key_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.enc_pub_key_free(app, handle, user_data, _enc_pub_key_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _enc_secret_key_new(self, app, data, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, AsymSecretKey*, [any], [function], [custom ffi lib]
        App* app, AsymSecretKey* data, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, EncryptSecKeyHandle sk_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,EncryptSecKeyHandle)")
    def _enc_secret_key_new_o_cb(user_data ,result ,sk_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,sk_h)


    safenetLib.enc_secret_key_new(app, data, user_data, _enc_secret_key_new_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _enc_secret_key_get(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, EncryptSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, EncryptSecKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AsymSecretKey* sec_enc_key)
    """
    @ffi.callback("void(void* ,FfiResult* ,AsymSecretKey*)")
    def _enc_secret_key_get_o_cb(user_data ,result ,sec_enc_key):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,sec_enc_key)


    safenetLib.enc_secret_key_get(app, handle, user_data, _enc_secret_key_get_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _enc_secret_key_free(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, EncryptSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, EncryptSecKeyHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _enc_secret_key_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.enc_secret_key_free(app, handle, user_data, _enc_secret_key_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sign(self, app, data, data_len, sign_sk_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, uint8_t*, uintptr_t, SignSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, uint8_t* data, uintptr_t data_len, SignSecKeyHandle sign_sk_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* signed_data, uintptr_t signed_data_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _sign_o_cb(user_data ,result ,signed_data ,signed_data_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,signed_data ,signed_data_len)


    safenetLib.sign(app, data, data_len, sign_sk_h, user_data, _sign_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _verify(self, app, signed_data, signed_data_len, sign_pk_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, uint8_t*, uintptr_t, SignPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, uint8_t* signed_data, uintptr_t signed_data_len, SignPubKeyHandle sign_pk_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* verified_data, uintptr_t verified_data_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _verify_o_cb(user_data ,result ,verified_data ,verified_data_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,verified_data ,verified_data_len)


    safenetLib.verify(app, signed_data, signed_data_len, sign_pk_h, user_data, _verify_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encrypt(self, app, data, data_len, public_key_h, secret_key_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, uint8_t*, uintptr_t, EncryptPubKeyHandle, EncryptSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, uint8_t* data, uintptr_t data_len, EncryptPubKeyHandle public_key_h, EncryptSecKeyHandle secret_key_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* ciphertext, uintptr_t ciphertext_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _encrypt_o_cb(user_data ,result ,ciphertext ,ciphertext_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,ciphertext ,ciphertext_len)


    safenetLib.encrypt(app, data, data_len, public_key_h, secret_key_h, user_data, _encrypt_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _decrypt(self, app, data, data_len, public_key_h, secret_key_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, uint8_t*, uintptr_t, EncryptPubKeyHandle, EncryptSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, uint8_t* data, uintptr_t data_len, EncryptPubKeyHandle public_key_h, EncryptSecKeyHandle secret_key_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* plaintext, uintptr_t plaintext_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _decrypt_o_cb(user_data ,result ,plaintext ,plaintext_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,plaintext ,plaintext_len)


    safenetLib.decrypt(app, data, data_len, public_key_h, secret_key_h, user_data, _decrypt_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _encrypt_sealed_box(self, app, data, data_len, public_key_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, uint8_t*, uintptr_t, EncryptPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, uint8_t* data, uintptr_t data_len, EncryptPubKeyHandle public_key_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* ciphertext, uintptr_t ciphertext_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _encrypt_sealed_box_o_cb(user_data ,result ,ciphertext ,ciphertext_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,ciphertext ,ciphertext_len)


    safenetLib.encrypt_sealed_box(app, data, data_len, public_key_h, user_data, _encrypt_sealed_box_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _decrypt_sealed_box(self, app, data, data_len, public_key_h, secret_key_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, uint8_t*, uintptr_t, EncryptPubKeyHandle, EncryptSecKeyHandle, [any], [function], [custom ffi lib]
        App* app, uint8_t* data, uintptr_t data_len, EncryptPubKeyHandle public_key_h, EncryptSecKeyHandle secret_key_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* plaintext, uintptr_t plaintext_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _decrypt_sealed_box_o_cb(user_data ,result ,plaintext ,plaintext_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,plaintext ,plaintext_len)


    safenetLib.decrypt_sealed_box(app, data, data_len, public_key_h, secret_key_h, user_data, _decrypt_sealed_box_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _sha3_hash(self, data, data_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        uint8_t* data, uintptr_t data_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* hash, uintptr_t hash_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t)")
    def _sha3_hash_o_cb(user_data ,result ,hash ,hash_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,hash ,hash_len)


    safenetLib.sha3_hash(data, data_len, user_data, _sha3_hash_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _generate_nonce(self, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        [any], [function], [custom ffi lib]
        void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, AsymNonce* nonce)
    """
    @ffi.callback("void(void* ,FfiResult* ,AsymNonce*)")
    def _generate_nonce_o_cb(user_data ,result ,nonce):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,nonce)


    safenetLib.generate_nonce(user_data, _generate_nonce_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _cipher_opt_new_plaintext(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, CipherOptHandle handle)
    """
    @ffi.callback("void(void* ,FfiResult* ,CipherOptHandle)")
    def _cipher_opt_new_plaintext_o_cb(user_data ,result ,handle):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,handle)


    safenetLib.cipher_opt_new_plaintext(app, user_data, _cipher_opt_new_plaintext_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _cipher_opt_new_symmetric(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, CipherOptHandle handle)
    """
    @ffi.callback("void(void* ,FfiResult* ,CipherOptHandle)")
    def _cipher_opt_new_symmetric_o_cb(user_data ,result ,handle):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,handle)


    safenetLib.cipher_opt_new_symmetric(app, user_data, _cipher_opt_new_symmetric_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _cipher_opt_new_asymmetric(self, app, peer_encrypt_key_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, EncryptPubKeyHandle, [any], [function], [custom ffi lib]
        App* app, EncryptPubKeyHandle peer_encrypt_key_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, CipherOptHandle handle)
    """
    @ffi.callback("void(void* ,FfiResult* ,CipherOptHandle)")
    def _cipher_opt_new_asymmetric_o_cb(user_data ,result ,handle):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,handle)


    safenetLib.cipher_opt_new_asymmetric(app, peer_encrypt_key_h, user_data, _cipher_opt_new_asymmetric_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _cipher_opt_free(self, app, handle, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, CipherOptHandle, [any], [function], [custom ffi lib]
        App* app, CipherOptHandle handle, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _cipher_opt_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.cipher_opt_free(app, handle, user_data, _cipher_opt_free_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entries_new(self, app, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, [any], [function], [custom ffi lib]
        App* app, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataEntriesHandle entries_h)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataEntriesHandle)")
    def _mdata_entries_new_o_cb(user_data ,result ,entries_h):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,entries_h)


    safenetLib.mdata_entries_new(app, user_data, _mdata_entries_new_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entries_insert(self, app, entries_h, key, key_len, value, value_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntriesHandle, uint8_t*, uintptr_t, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        App* app, MDataEntriesHandle entries_h, uint8_t* key, uintptr_t key_len, uint8_t* value, uintptr_t value_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_entries_insert_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_entries_insert(app, entries_h, key, key_len, value, value_len, user_data, _mdata_entries_insert_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entries_len(self, app, entries_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntriesHandle, [any], [function], [custom ffi lib]
        App* app, MDataEntriesHandle entries_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uintptr_t len)
    """
    @ffi.callback("void(void* ,FfiResult* ,uintptr_t)")
    def _mdata_entries_len_o_cb(user_data ,result ,len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,len)


    safenetLib.mdata_entries_len(app, entries_h, user_data, _mdata_entries_len_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entries_get(self, app, entries_h, key, key_len, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntriesHandle, uint8_t*, uintptr_t, [any], [function], [custom ffi lib]
        App* app, MDataEntriesHandle entries_h, uint8_t* key, uintptr_t key_len, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, uint8_t* content, uintptr_t content_len, uint64_t version)
    """
    @ffi.callback("void(void* ,FfiResult* ,uint8_t* ,uintptr_t ,uint64_t)")
    def _mdata_entries_get_o_cb(user_data ,result ,content ,content_len ,version):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,content ,content_len ,version)


    safenetLib.mdata_entries_get(app, entries_h, key, key_len, user_data, _mdata_entries_get_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_list_entries(self, app, entries_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntriesHandle, [any], [function], [custom ffi lib]
        App* app, MDataEntriesHandle entries_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result, MDataEntry* entries, uintptr_t entries_len)
    """
    @ffi.callback("void(void* ,FfiResult* ,MDataEntry* ,uintptr_t)")
    def _mdata_list_entries_o_cb(user_data ,result ,entries ,entries_len):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result ,entries ,entries_len)


    safenetLib.mdata_list_entries(app, entries_h, user_data, _mdata_list_entries_o_cb)



@safeUtils.safeThread(timeout=5,queue=self.queue)
def _mdata_entries_free(self, app, entries_h, user_data, o_cb=None, safenetLib=self.lib.safe_app):
    """
        App*, MDataEntriesHandle, [any], [function], [custom ffi lib]
        App* app, MDataEntriesHandle entries_h, void* user_data

        > callback functions:
        (*o_cb)(void* user_data, FfiResult* result)
    """
    @ffi.callback("void(void* ,FfiResult*)")
    def _mdata_entries_free_o_cb(user_data ,result):
        self.safeUtils.checkResult(result)
        self.queue.put('gotResult')
        if o_cb:
            o_cb(user_data ,result)


    safenetLib.mdata_entries_free(app, entries_h, user_data, _mdata_entries_free_o_cb)